In [ ]:
import torch
from torch import nn
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingLR, MultiStepLR
import torchvision
from torchvision import transforms, utils, datasets

import os
import sys
import cv2
import PIL
import numpy as np
import random
import logging
import gdown
from zipfile import ZipFile
import matplotlib.pyplot as plt

#if '/opt/ros/kinetic/lib/python2.7/dist-packages' in sys.path:
#    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')

## Parameters

In [ ]:
BATCH_SIZE = 30
NUM_WROKERS = 8
INPUT_IMG_SIZE = (101, 101)
EPOCH = 3000               # train the training data n times, to save time, we just train 1 epoch
LR = 0.01               # learning rate
DATASET_ROOT = './dataset/joystick'
MODELS_ROOT = './models'
CLASSES = np.loadtxt('./class_id.txt', str, delimiter='\n')

## Useful functions: 
dataset visualization and image loading function

In [ ]:
def vis_img(batch_data):   
    # show images
    imgs = torchvision.utils.make_grid(batch_data)
    imgs = imgs / 2 + 0.5     # unnormalize
    npimgs = imgs.numpy()
    plt.rcParams['figure.figsize'] = [12, 5]
    plt.imshow(np.transpose(npimgs, (1, 2, 0)))

In [ ]:
def load_images_from_folder(folder, classes):
    img_paths = []
    labels = []
    for class_id, class_name in enumerate(classes):
        class_folder = os.path.join(folder, class_name)
        for filename in os.listdir(class_folder):
            filename.lower().endswith(('.png', '.jpg', '.jpeg'))
            img_paths.append(os.path.join(class_folder, filename))
            labels.append(class_id)
    return img_paths, labels

## Custom pytorch dataset class for Trailnet (load dataset from txt)

In [ ]:
class TrailnetDataset(Dataset):
    def __init__(self, datalist_filename):
        classes = CLASSES
        
        self.img_list = []
        self.label_list = []
        
        txt = np.loadtxt(datalist_filename, str, delimiter='\n')
        for line in txt:
            p, l  = line.split()
            self.img_list.append(p)
            self.label_list.append(l)
        
        print( '********** Dataset Info start **********\n')
        print('Source: ', datalist_filename) 
        print('Output classes: ', classes) 
        print( 'Amount of images: ', len(txt))
        print('\n*********** Dataset Info end ***********\n') 

        

        
        self.data_transform = transforms.Compose([ 
                                transforms.Resize(INPUT_IMG_SIZE), \
                                transforms.ToTensor(), \
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], \
                                                     std=[1, 1, 1]), \
                                ])
        
    def __len__(self):
        return len(self.img_list)
    
    def collect_folders_from_dataset(self, dataset_root, classes):
        # Implement by BFS
        search_list = [dataset_root, ]
        dataset_folders = [] 
        while len(search_list) != 0:
            root = search_list.pop(0)
            if set(os.listdir(root)) == set(classes):
                dataset_folders.append(root)
            else:
                for folder in os.listdir(root):
                    path = os.path.join(root, folder)
                    if os.path.isdir(path):
                        search_list.append(path)
        return dataset_folders

    def __getitem__(self, index):
        'Generates one sample of data'
        # print self.img_list[index]
        # Select sample, then load data and get label
        path = self.img_list[index]
        img_raw = self.default_loader(path)
        x = self.data_transform(img_raw)
        z = float(self.label_list[index])
        y = int((z/8.5+1)*7)
  
        return x, y, path
    
    def pil_loader(self, path):
        with open(path, 'rb') as f:
            with PIL.Image.open(f) as img:
                return img.convert('RGB')

    def accimage_loader(self, path):
        try:
            return accimage.Image(path)
        except IOError:
            # Potentially a decoding problem, fall back to PIL.Image
            return pil_loader(path)

    def default_loader(self, path):
        if torchvision.get_image_backend() == 'accimage':
            return self.accimage_loader(path)
        else:
            return self.pil_loader(path)

## Train / validation dataset declaration 

In [ ]:
train_dataset = TrailnetDataset(datalist_filename='./dataset/train.txt')
val_dataset = TrailnetDataset(datalist_filename='./dataset/test.txt')

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WROKERS, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WROKERS, shuffle=False) 

## Training and Testing function

In [ ]:
def train(loader, model, criterion, optimizer, device, debug_steps=100, epoch=-1):
    model.train(True)
    running_loss = 0.0
    running_regression_loss = 0.0
    running_classification_loss = 0.0
    model.to(device)
    for i, data in enumerate(loader):
        images, labels, _path = data
        images = images.to(device)
        labels = labels.to(device)
        
        

        optimizer.zero_grad()
        
        outputs = model(images)

 
        loss = criterion(outputs, labels)  # TODO CHANGE BOXES

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i and i % debug_steps == 0:
            avg_loss = loss / debug_steps
            # logging.info("Epoch: {}, Step: {}, Average Loss: {:.4f}".format(epoch, i, avg_loss))
            print( "Epoch: {}, Step: {}, Average Loss: {:.6f}".format(epoch, i, avg_loss))
            running_loss = 0.0
        

def test(loader, model, criterion, device):
    model.eval()
    running_loss = 0.0
    num = 0
    for _, data in enumerate(loader):
        images, labels, _path = data
        images = images.to(device)
        labels = labels.to(device)
        num += 1

        with torch.no_grad():
            outputs = model(images)
            loss = criterion(outputs, labels)

        running_loss += loss.item()
    return running_loss / num

## AlexNet 15 class output model

In [ ]:
class AlexNet(nn.Module):

    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 15),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


## TrailNet 15 class output model

In [ ]:
# class TrailNet15Class(nn.Module):
#     def __init__(self):
#         super(TrailNet15Class, self).__init__()
#         # 1 input image channel, 6 output channels, 5x5 square convolution
#         # kernel
#         self.conv1 = nn.Conv2d(3, 32, 4)
#         self.pool1 = nn.MaxPool2d((2, 2), stride=2)
#         self.conv2 = nn.Conv2d(32, 32, 4)
#         self.pool2 = nn.MaxPool2d((2, 2), stride=2)
#         self.conv3 = nn.Conv2d(32, 32, 4)
#         self.pool3 = nn.MaxPool2d((2, 2), stride=2)
#         self.conv4 = nn.Conv2d(32, 32, 4, padding=(2, 2))
#         self.pool4 = nn.MaxPool2d((2, 2), stride=2)
#         # an affine operation: y = Wx + b
#         self.fc1 = nn.Linear(800, 200)
#         self.fc2 = nn.Linear(200, 15)

#     def forward(self, x):
#         x = self.pool1(self.conv1(x))
#         x = self.pool2(self.conv2(x))
#         x = self.pool3(self.conv3(x)) 
#         x = self.pool4(self.conv4(x))    
#         # print 'x size: ', x.size()   
#         x = x.view(-1, self.num_flat_features(x))
#         # print 'x size: ', x.size()   
#         x = self.fc1(x)
#         x = self.fc2(x)
#         return x

#     def num_flat_features(self, x):
#         size = x.size()[1:]  # all dimensions except the batch dimension
#         num_features = 1
#         for s in size:
#             num_features *= s
#         return num_features

In [ ]:
#model = TrailNet15Class()
model = AlexNet()

## Training process

In [ ]:
last_epoch = -1
validation_epochs = 3

params = model.parameters()



checkpoint_folder = './models'
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params, lr=1e-3, momentum=0.9, weight_decay=5e-4)
scheduler = CosineAnnealingLR(optimizer, 120, last_epoch=last_epoch)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for epoch in range(last_epoch + 1, EPOCH):
    scheduler.step()
    train(train_loader, model, criterion, optimizer, \
          device=DEVICE, debug_steps=50, epoch=epoch)

    if epoch % validation_epochs == 0 or epoch == EPOCH - 1:
        val_loss = test(val_loader, model, criterion, DEVICE)
        # logging.info("Epoch: {}, ".format(epoch) + \
        #            "Validation Loss: {:.4f}".format(val_loss))
        print("Epoch: {}, Validation Loss: {:.6f}".format(epoch, val_loss)) 
        model_path = os.path.join(checkpoint_folder, \
                                  "{}_epoch{}_loss{:.5f}.pth".format('AlexNet_15class', epoch, val_loss))
        torch.save(model.state_dict(), model_path)
        # logging.info("Saved model {}".format(model_path))
